# load modules and functions

In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split

# load and explore trainging dataset

In [3]:
data = pd.read_csv("house-prices-advanced-regression-techniques/train.csv")
label = data["SalePrice"]
data = data.drop(columns = ["SalePrice"])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 80 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

# based on dataset description:
- "MSSubClass" column actually contains parallel levels, it needs to be converted to categorical;
- "Alley", "FireplaceQu", "PoolQC", "Fence" and "MiscFeature" columns contain few information, which need to be dropped;
- "LotShape", "Utilities", "LandSlope", "ExterQual", "ExterCond", "BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "BsmtFinType2", "HeatingQC", "CentralAir", "Electrical", "KitchenQual", "Functional", "GarageFinish", "GarageQual" and "GarageCond" columns contain levels, which need to be converted to numeric.

then all the numerical columns are normalize, all the categorical columns are one hot encoded.

In [9]:
def process(x):
    # there are several columns are almost empty, remove them. also remove ID
    x = x.drop(columns=["Id", "Alley", "FireplaceQu", "PoolQC", "Fence", "MiscFeature"])
    # "MSSubClass" actually have parallel levels. turn it to categorical
    x["MSSubClss"] = x["MSSubClass"].astype("object")
    
    for i in range(len(x["LotShape"])):
        if x["LotShape"][i] == "Reg":
            x["LotShape"][i] = 3
        elif x["LotShape"][i] == "IR1":
            x["LotShape"][i] = 2
        elif x["LotShape"][i] == "IR2":
            x["LotShape"][i] = 1
        else:
            x["LotShape"][i] = 0
        
    for i in range(len(x["Utilities"])):
        if x["Utilities"][i] == "AllPub":
            x["Utilities"][i] = 3
        elif x["Utilities"][i] == "NoSewr":
            x["Utilities"][i] = 2
        elif x["Utilities"][i] =="NoSeWa":
            x["Utilities"][i] = 2
        else:
            x["Utilities"][i] = 1

    for i in range(len(x["LandSlope"])):
        if x["LandSlope"][i] == "Gtl":
            x["LandSlope"][i] = 2
        elif x["LandSlope"][i] == "Mod":
            x["LandSlope"][i] = 1
        else:
            x["LandSlope"][i] = 0
        
    for i in range(len(x["ExterQual"])):
        if x["ExterQual"][i] == "Ex":
            x["ExterQual"][i] = 4
        elif x["ExterQual"][i] == "Gd":
            x["ExterQual"][i] = 3
        elif x["ExterQual"][i] == "TA":
            x["ExterQual"][i] = 2
        elif x["ExterQual"][i] == "Fa":
            x["ExterQual"][i] = 1
        else:
            x["ExterQual"][i] = 0
        
    for i in range(len(x["ExterCond"])):
        if x["ExterCond"][i] == "Ex":
            x["ExterCond"][i] = 4
        elif x["ExterCond"][i] == "Gd":
            x["ExterCond"][i] = 3
        elif x["ExterCond"][i] == "TA":
            x["ExterCond"][i] = 2
        elif x["ExterCond"][i] == "Fa":
            x["ExterCond"][i] = 1
        else:
            x["ExterCond"][i] = 0
        
    for i in range(len(x["BsmtQual"])):
        if x["BsmtQual"][i] == "Ex":
            x["BsmtQual"][i] = 5
        elif x["BsmtQual"][i] == "Gd":
            x["BsmtQual"][i] = 4
        elif x["BsmtQual"][i] == "TA":
            x["BsmtQual"][i] = 3
        elif x["BsmtQual"][i] == "Fa":
            x["BsmtQual"][i] = 2
        elif x["BsmtQual"][i] == "Po":
            x["BsmtQual"][i] = 1
        else:
            x["BsmtQual"][i] = 0
        
    for i in range(len(x["BsmtCond"])):
        if x["BsmtCond"][i] == "Ex":
            x["BsmtCond"][i] = 5
        elif x["BsmtCond"][i] == "Gd":
            x["BsmtCond"][i] = 4
        elif x["BsmtCond"][i] == "TA":
            x["BsmtCond"][i] = 3
        elif x["BsmtCond"][i] == "Fa":
            x["BsmtCond"][i] = 2
        elif x["BsmtCond"][i] == "Po":
            x["BsmtCond"][i] = 1
        else:
            x["BsmtCond"][i] = 0

    for i in range(len(x["BsmtExposure"])):
        if x["BsmtExposure"][i] == "Gd":
            x["BsmtExposure"][i] = 4
        elif x["BsmtExposure"][i] == "Av":
            x["BsmtExposure"][i] = 3
        elif x["BsmtExposure"][i] == "Mn":
            x["BsmtExposure"][i] = 2
        elif x["BsmtExposure"][i] == "No":
            x["BsmtExposure"][i] = 1
        else:
            x["BsmtExposure"][i] = 0

    for i in range(len(x["BsmtFinType1"])):
        if x["BsmtFinType1"][i] == "GLQ":
            x["BsmtFinType1"][i] = 6
        elif x["BsmtFinType1"][i] == "ALQ":
            x["BsmtFinType1"][i] = 5
        elif x["BsmtFinType1"][i] == "BLQ":
            x["BsmtFinType1"][i] = 4
        elif x["BsmtFinType1"][i] == "Rec":
            x["BsmtFinType1"][i] = 3
        elif x["BsmtFinType1"][i] == "LwQ":
            x["BsmtFinType1"][i] = 2
        elif x["BsmtFinType1"][i] == "Unf":
            x["BsmtFinType1"][i] = 1
        else:
            x["BsmtFinType1"][i] = 0

    for i in range(len(x["BsmtFinType2"])):
        if x["BsmtFinType2"][i] == "GLQ":
            x["BsmtFinType2"][i] = 6
        elif x["BsmtFinType2"][i] == "ALQ":
            x["BsmtFinType2"][i] = 5
        elif x["BsmtFinType2"][i] == "BLQ":
            x["BsmtFinType2"][i] = 4
        elif x["BsmtFinType2"][i] == "Rec":
            x["BsmtFinType2"][i] = 3
        elif x["BsmtFinType2"][i] == "LwQ":
            x["BsmtFinType2"][i] = 2
        elif x["BsmtFinType2"][i] == "Unf":
            x["BsmtFinType2"][i] = 1
        else:
            x["BsmtFinType2"][i] = 0

    for i in range(len(x["HeatingQC"])):
        if x["HeatingQC"][i] == "Ex":
            x["HeatingQC"][i] = 4
        elif x["HeatingQC"][i] == "Gd":
            x["HeatingQC"][i] = 3
        elif x["HeatingQC"][i] == "TA":
            x["HeatingQC"][i] = 2
        elif x["HeatingQC"][i] == "Fa":
            x["HeatingQC"][i] = 1
        else:
            x["HeatingQC"][i] = 0

    for i in range(len(x["CentralAir"])):
        if x["CentralAir"][i] == "N":
            x["CentralAir"][i] = 0
        else:
            x["CentralAir"][i] = 1

    for i in range(len(x["Electrical"])):
        if x["Electrical"][i] == "SBrkr":
            x["Electrical"][i] = 4
        elif x["Electrical"][i] == "FuseA":
            x["Electrical"][i] = 3
        elif x["Electrical"][i] == "FuseF":
            x["Electrical"][i] = 2
        elif x["Electrical"][i] == "FuseP":
            x["Electrical"][i] = 1
        else:
            x["Electrical"][i] = 0

    for i in range(len(x["KitchenQual"])):
        if x["KitchenQual"][i] == "Ex":
            x["KitchenQual"][i] = 4
        elif x["KitchenQual"][i] == "Gd":
            x["KitchenQual"][i] = 3
        elif x["KitchenQual"][i] == "TA":
            x["KitchenQual"][i] = 2
        elif x["KitchenQual"][i] == "Fa":
            x["KitchenQual"][i] = 1
        else:
            x["KitchenQual"][i] = 0

    for i in range(len(x["Functional"])):
        if x["Functional"][i] == "Typ":
            x["Functional"][i] = 7
        elif x["Functional"][i] == "Min1":
            x["Functional"][i] = 6
        elif x["Functional"][i] == "Min2":
            x["Functional"][i] = 5
        elif x["Functional"][i] == "Mod":
            x["Functional"][i] = 4
        elif x["Functional"][i] == "Maj1":
            x["Functional"][i] = 3
        elif x["Functional"][i] == "Maj2":
            x["Functional"][i] = 2
        elif x["Functional"][i] == "Sev":
            x["Functional"][i] = 1
        else:
            x["Functional"][i] = 0

    for i in range(len(x["GarageFinish"])):
        if x["GarageFinish"][i] == "Fin":
            x["GarageFinish"][i] = 3
        elif x["GarageFinish"][i] == "RFn":
            x["GarageFinish"][i] = 2
        elif x["GarageFinish"][i] == "Unf":
            x["GarageFinish"][i] = 1
        else:
            x["GarageFinish"][i] = 0

    for i in range(len(x["GarageQual"])):
        if x["GarageQual"][i] == "Ex":
            x["GarageQual"][i] = 5
        elif x["GarageQual"][i] == "Gd":
            x["GarageQual"][i] = 4
        elif x["GarageQual"][i] == "TA":
            x["GarageQual"][i] = 3
        elif x["GarageQual"][i] == "Fa":
            x["GarageQual"][i] = 2
        elif x["GarageQual"][i] == "Po":
            x["GarageQual"][i] = 1
        else:
            x["GarageQual"][i] = 0

    for i in range(len(x["GarageCond"])):
        if x["GarageCond"][i] == "Ex":
            x["GarageCond"][i] = 5
        elif x["GarageCond"][i] == "Gd":
            x["GarageCond"][i] = 4
        elif x["GarageCond"][i] == "TA":
            x["GarageCond"][i] = 3
        elif x["GarageCond"][i] == "Fa":
            x["GarageCond"][i] = 2
        elif x["GarageCond"][i] == "Po":
            x["GarageCond"][i] = 1
        else:
            x["GarageCond"][i] = 0
    
    # convert these columns to numeric
    columns = ["LotShape", "Utilities", "LandSlope", "ExterQual", "ExterCond",
          "BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "KitchenQual",
          "BsmtFinType2", "HeatingQC", "CentralAir", "Electrical",
          "Functional", "GarageFinish", "GarageQual", "GarageCond"]
    x[columns] = x[columns].astype("int")
    
    # select categorical and numerical column names
    categorical = list(x.select_dtypes(include="object").columns)
    numeric = [col for col in x.columns if col not in categorical]
    
    # select categorical and numerical columns
    cat = x[categorical]
    num = x[numeric]

    # impute cateorical data
    imp_cat = SimpleImputer(missing_values=np.NaN, strategy="most_frequent")
    cat_imp = imp_cat.fit_transform(cat)
    ohc = OneHotEncoder(sparse=False)
    cat_ready = pd.DataFrame(ohc.fit_transform(cat_imp))

    # impute numerical data
    imp_num = SimpleImputer(missing_values=np.NaN, strategy="mean")
    num_imp = imp_num.fit_transform(num)
    mms = MinMaxScaler()
    num_ready = pd.DataFrame(mms.fit_transform(num_imp))

    data_ready = pd.concat([cat_ready, num_ready], axis=1)
    
    return data_ready

In [10]:
data_full = process(data)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.p

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:175: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:177: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:183: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

Se

# define a funcion to calculate RMSE

In [13]:
def rmse(true, test):
    true = np.array(true)
    test = np.array(test)
    rmse = np.sqrt(sum((true - test) ** 2) / len(true))
    return rmse

# split data

In [16]:
X_train, y_train, X_test, y_test = train_test_split(data_full, label, test_size=0.2, random_state=1988)

# classic linear regression

In [17]:
lr = LinearRegression()
lr.fit(X_train, X_test)
pred_lr = lr.predict(y_train)
print(rmse(y_test, pred_lr))

5938539295682489.0


# lasso regression with grid of alpha values

In [18]:
param_grid = {"alpha":np.linspace(100, 10000, 100)}

ls = Lasso()
grid_ls = GridSearchCV(ls, param_grid=param_grid, cv=10)
grid_ls.fit(X_train, X_test)
pred_ls = grid_ls.predict(y_train)
print(grid_ls.best_params_)
print(grid_ls.best_score_)
print(rmse(y_test, pred_ls))

{'alpha': 300.0}
0.8022045623433333
27020.399739126533


# ridge regression with grid of alpha values

In [19]:
rg = Ridge()
grid_rg = GridSearchCV(rg, param_grid=param_grid, cv=10)
grid_rg.fit(X_train, X_test)
pred_rg = grid_rg.predict(y_train)
print(grid_rg.best_params_)
print(grid_rg.best_score_)
print(rmse(y_test, pred_rg))

{'alpha': 100.0}
0.7458865824566262
36422.57866981436


# lasso regression outperforms others. 
# try it on test dataset.

In [20]:
test = pd.read_csv("house-prices-advanced-regression-techniques/test.csv")
test_full = process(test)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.p

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See th

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:171: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:175: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

Se

In [21]:
pred = grid_ls.predict(test_full)

ValueError: shapes (1459,213) and (225,) not aligned: 213 (dim 1) != 225 (dim 0)

In [28]:
test.drop(columns=["Id", "Alley", "FireplaceQu", "PoolQC", "Fence", "MiscFeature"]).describe(include="object").T

,count,unique,top,freq
MSZoning,1455,5,RL,1114
Street,1459,2,Pave,1453
LotShape,1459,4,Reg,934
LandContour,1459,4,Lvl,1311
Utilities,1457,1,AllPub,1457
LotConfig,1459,5,Inside,1081
LandSlope,1459,3,Gtl,1396
Neighborhood,1459,25,NAmes,218
Condition1,1459,9,Norm,1251
Condition2,1459,5,Norm,1444


In [29]:
data.drop(columns=["Id", "Alley", "FireplaceQu", "PoolQC", "Fence", "MiscFeature"]).describe(include="object").T

,count,unique,top,freq
MSZoning,1460,5,RL,1151
Street,1460,2,Pave,1454
LotShape,1460,4,Reg,925
LandContour,1460,4,Lvl,1311
Utilities,1460,2,AllPub,1459
LotConfig,1460,5,Inside,1052
LandSlope,1460,3,Gtl,1382
Neighborhood,1460,25,NAmes,225
Condition1,1460,9,Norm,1260
Condition2,1460,8,Norm,1445
